# Building a Spam Filter with Naive Bayes

**Project Goal**: Design a filter to detect spam SMS messages.

**Dataset**: The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

# Part 1: Exploratory Data Analysis

In [36]:
import pandas as pd

In [37]:
df = pd.read_csv('SMSSpamCollection',sep='\t',header=None,names=['Label', 'SMS'])

In [38]:
print(df.columns)

Index(['Label', 'SMS'], dtype='object')


In [39]:
print(str(len(df)))

5572


In [59]:
pd.set_option('display.max_colwidth', -1)

In [60]:
df.iloc[1,1]

'Ok lar... Joking wif u oni...'

In [49]:
df['SMS'].str.replace('\W',' ')
print(df.iloc[:1,:2])

  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...


In [51]:
df.SMS.str.replace('[^a-zA-Z]',' ')
print(df.iloc[0,1])

  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...


## Part 2: Data Preprocessing and Cleaning

Well need to do some encoding to map the dataset from using strings as messages, to using quantitative frequency measurements for how many times a given word was present in the message.

This will enable us to calculate occurrence frequencies for individual words across the data set and then eventually probabilities.

We'll also need to remove punctuation and standardize capitalization across all of the messages to simplify string matching for words later.

In [31]:
df['SMS'].str.replace('\W',' ',regex=True)
df['SMS'] = df['SMS'].str.lower()

In [32]:
print(df.head(3))

  Label                                                SMS
0   ham  go until jurong point, crazy.. available only ...
1   ham                      ok lar... joking wif u oni...
2  spam  free entry in 2 a wkly comp to win fa cup fina...


In [33]:
df['SMS'] = df['SMS'].str.split()

In [34]:
print(df.head(3))

  Label                                                SMS
0   ham  [go, until, jurong, point,, crazy.., available...
1   ham               [ok, lar..., joking, wif, u, oni...]
2  spam  [free, entry, in, 2, a, wkly, comp, to, win, f...


In [35]:
vocab_list = []
for message in df['SMS']:
    for word in message:
        vocab_list.append(word)
        
vocabulary = list(set(vocab_list))
print(vocabulary)

['paces,', 'shrek', 'game!', 'enough.', 'oral.', 'dead...', 'www.80488.biz', 'mat', 'howda', 'designation', 'vth', 'rebel!', 'bag', 'scenario', 'fathima', 'peril', 'lodge', 'good?', 'came', 'owns', 'archive.', '08718726978', 'weaseling', 'welcome!', 'algebra', 'demand', 'evr', 'rcb.battle', 'poem:', '1.20', 'hitter.anyway', 'prakasamanu', '09058097218', 'semester.', 'sickness', '***************', 'james', 'gre', 'available', '14thmarch.', 'vday', 'hmph.', 'korean', 'mob.', 'line:', 'http://www.bubbletext.com', 'week??', 'wannatell', 'freinds', 'senthil', 'anyways', 'freely,', 'gm+gn+ge+gn:)', 'ps', 'poly/200p', 'spacebucks', 'cheque.', '4u', 'instead!', 'rude', '0789xxxxxxx.', 'province', 'tuition...', "you're", 'jet', 'overdid', 'being', 'serving.', 'celebrate.', 'lives', 'free..', 'freaking', 'lily', 'dorm.', 'vargu', 'gandhipuram', 'convenience.', 'pop', 'face.', 'watch,', 'neville?', 'personally', "ny's", 'newport', 'egg.', 'also.or', 'logo&pic', 'reffering', '$50...', 'dump', 'ale

## Part 2: Dividing test and training data

* Reserving 20% of the data for testing, 80% for training the model
* We'll randomize the dataset before splitting

In [6]:
def print_label_totals(df,column,labels):
    for label in labels:
        total_rows = len(df)
        num_label = len(df[df[column] == label])
        print(str(num_label)+" out of "+str(total_rows)+" are label: "+label+" - "+str((num_label/total_rows)*100)+"%")

In [9]:
randomized_df = df.sample(frac=1,random_state=1)
num_training_rows = int(len(df)*.8)
training_set = df.iloc[:num_training_rows]
test_set = df.iloc[num_training_rows:]
print_label_totals(training_set,'Label',['ham','spam'])
print_label_totals(test_set,'Label',['ham','spam'])

3855 out of 4457 are label: ham - 86.49315683194975%
602 out of 4457 are label: spam - 13.506843168050258%
970 out of 1115 are label: ham - 86.99551569506725%
145 out of 1115 are label: spam - 13.004484304932735%


## Part 3: Training the Naive Bayes Algorithm

### Theoretical basis

We will use the following equations:
\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}